In [1]:
import numpy as np
import math
import scipy.optimize
import time

np.random.seed(5)
np.set_printoptions(precision=4)

In [2]:
def generateModel(locN, stageN):
    # Generate the prior distribution vector
    priorVec = np.random.random(locN)
    priorVec /= priorVec.sum()
    print('The prior probability vector is:\n' + str(priorVec) + '\n')

    # Generate the Markov transition probability matrix (left stochastic matrix; each column sums up to one)
    tranProbMat = np.random.rand(locN, locN)
    tranProbMat /= tranProbMat.sum(axis=0)[None,:]
    print('The Markov transition probability matrix (x=Px) is:\n' + str(tranProbMat) + '\n')

    # Generate the probability vector of successful detections (between 0.5 to 1.0)
    detectVec = 1 - 0.5 * np.random.random(locN)
    print('The probability vector of successful detections is:\n' + str(detectVec) + '\n')

    # Compute the probability vector of missed detections
    missVec = 1 - detectVec
    print('The probability vector of failed detections is:\n' + str(missVec) + '\n')
    
    # Return model
    return priorVec, tranProbMat, detectVec, missVec

In [3]:
# agentN - number of agents
agentN = 20
# locN - number of locations
locN = 500
# stageN - number of stages
stageN = 500

#q = [ 0.0255, 0.2603, 0.1463, 0.2415, 0.3264 ]
#missVec = [ 0.1024, 0.2454, 0.1862, 0.2387, 0.1829 ]

[priorVec, tranProbMat, detectVec, missVec] = generateModel(locN, stageN)
q = priorVec

The prior probability vector is:
[  8.8772e-04   3.4819e-03   8.2664e-04   3.6734e-03   1.9531e-03
   2.4463e-03   3.0627e-03   2.0731e-03   1.1869e-03   7.5067e-04
   3.2287e-04   2.9529e-03   1.7647e-03   6.3306e-04   3.5187e-03
   1.0960e-03   1.6565e-03   1.1840e-03   2.5144e-03   2.3187e-03
   2.3990e-03   1.0630e-03   1.1384e-03   1.0141e-03   1.3099e-03
   5.7649e-04   6.6226e-04   3.8546e-03   3.8398e-03   7.5344e-04
   9.7198e-05   8.1798e-04   2.7986e-03   3.1172e-03   9.1706e-05
   2.3100e-03   6.5668e-06   2.0613e-03   2.5584e-03   3.9414e-03
   1.0361e-03   3.2091e-03   3.4809e-03   3.6899e-03   8.8543e-06
   1.8774e-03   3.9247e-03   1.5953e-03   3.2540e-03   2.1852e-03
   3.0825e-03   1.9392e-03   1.1641e-04   3.4600e-04   4.4569e-04
   1.0047e-03   3.8585e-03   2.5263e-03   3.2657e-03   2.2637e-03
   2.5407e-03   3.2467e-03   3.7057e-03   3.6494e-03   3.2983e-03
   3.7670e-04   1.4438e-03   1.4199e-04   2.1848e-03   3.1836e-03
   2.0451e-04   7.5445e-04   1.4615e-03   9

In [4]:
def relaxedSubprob(x):
    f = 0
    for k in range(locN):
        f += q[k] * math.pow(missVec[k], x[k])
    return f

def relaxedSubprob_deriv(x):
    f = []
    for k in range(locN):
        f.append(q[k] * math.log(missVec[k]) * math.pow(missVec[k], x[k]))
    return np.array(f)

print(relaxedSubprob_deriv(np.ones(locN)))

[ -3.2577e-04  -1.2322e-03  -1.3889e-04  -1.3445e-03  -6.5711e-04
  -8.7000e-04  -1.0662e-03  -5.5530e-04  -4.3475e-04  -1.6006e-04
  -1.1202e-04  -1.0374e-03  -6.4886e-04  -1.1227e-04  -1.2565e-03
  -4.0208e-04  -4.2749e-04  -4.3538e-04  -8.7866e-04  -8.4816e-04
  -7.1823e-04  -3.7070e-04  -4.1756e-04  -3.6323e-04  -4.5862e-04
  -8.7341e-05  -2.3945e-04  -1.4094e-03  -1.4117e-03  -2.1070e-04
  -3.4576e-05  -3.0092e-04  -9.3756e-04  -1.0956e-03  -3.1767e-05
  -8.0890e-04  -2.4115e-06  -7.0076e-04  -5.9664e-04  -1.4447e-03
  -2.1806e-04  -1.0322e-03  -6.3820e-04  -1.2252e-03  -1.4242e-06
  -6.8719e-04  -1.4362e-03  -5.8608e-04  -1.1147e-03  -5.1221e-04
  -1.1327e-03  -4.4753e-04  -4.2800e-05  -2.7020e-05  -1.5511e-04
  -1.7977e-04  -1.3889e-03  -9.2835e-04  -6.0603e-04  -8.2938e-04
  -9.3412e-04  -1.1105e-03  -1.3510e-03  -1.0851e-03  -6.1762e-04
  -1.1393e-04  -5.0070e-04  -4.9768e-05  -7.9517e-04  -1.1087e-03
  -6.4068e-05  -2.7704e-04  -5.3247e-04  -5.6784e-05  -8.6494e-04
  -5.0248e

In [5]:
cons = (
    {
        'type': 'eq',
        'fun': lambda x: np.array([sum(x) - agentN]),
        'jac': lambda x: np.ones(locN)
    },
    {
        'type': 'ineq',
        'fun': lambda x: np.array(x)
    }
)

In [6]:
st_time = time.time()
res = scipy.optimize.minimize(relaxedSubprob, np.zeros(locN),
                              jac=relaxedSubprob_deriv, method='SLSQP',
                              constraints=cons, options={'disp': True})
print('Time taken to run: '+ str(time.time() - st_time))
print('Optimal decision variables are: ' + str(res.x))

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.8740950727828528
            Iterations: 39
            Function evaluations: 39
            Gradient evaluations: 39
Time taken to run: 11.437943935394287
Optimal decision variables are: [ -3.2085e-17   2.6690e-02  -3.3602e-17  -1.2704e-16  -5.8177e-17
  -2.2320e-17  -4.2973e-17  -3.2193e-17   7.9193e-19  -6.3023e-18
  -1.5347e-17  -1.1737e-17  -1.6600e-17  -4.8761e-17   1.3304e-02
   1.7096e-18   8.7592e-18  -1.4906e-17  -2.5374e-17  -4.7383e-18
  -2.1626e-17  -1.9276e-18   3.2566e-17   3.7189e-17  -1.2855e-17
  -5.1186e-17  -5.2856e-17  -4.6594e-17  -7.0392e-18  -4.3009e-17
   1.5766e-17   3.1294e-17  -6.9507e-17   7.3292e-17   2.3296e-17
  -2.3317e-17   1.1727e-17  -6.2642e-17   1.2156e-01  -1.1384e-17
  -2.0376e-17   9.6053e-02   2.8119e-01   1.5380e-01  -4.5781e-17
   1.4681e-18  -1.3716e-17   6.1029e-18   3.7047e-02   4.9321e-02
  -1.2711e-17   9.8759e-04   1.6859e-17  -1.0682e-17   7.99